In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf 

rng = np.random.RandomState(0)

In [4]:
# Initialisierung von DataFrames und Data Cleaning
df = pd.read_csv('Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index
df.drop(['Betriebssystem_OHNE', 'Betriebssystem_Mac', 'Marke_Dell'], axis=1) # Mac und Marke_Apple sind identische Merkmale

df_noPrice = df.drop('Preis', axis=1)
imputed_simul_knn = pd.DataFrame(columns=['MSE KNN_1', 'MSE KNN_3', 'MSE KNN_5', 'SE KNN_1', 'SE KNN_3', 'SE KNN_5'])
imputed_stats_knn = pd.DataFrame(columns=['MSE KNN_1', 'MSE KNN_3', 'MSE KNN_5', 'SE KNN_1', 'SE KNN_3', 'SE KNN_5'], index = np.arange(0.1, 1, 0.1))

imputed_simul_ols = pd.DataFrame(columns=['MSE OLS', 'SE OLS'])
imputed_stats_ols = pd.DataFrame(columns=['MSE OLS', 'SE OLS'], index = np.arange(0.1, 1, 0.1))


# Skalierung (Standardisierung) von df_noPrice
col_names = df_noPrice.columns
scaler = StandardScaler().fit(df_noPrice.values)
df_noPrice = scaler.transform(df_noPrice.values)
df_noPrice = pd.DataFrame(df_noPrice, columns=col_names)


# Skalierung (Standardisierung) von df mit Preis (Preis ist unverändert)
df_std = df_noPrice.copy()
df_std.insert(0, 'Preis', df['Preis'])

In [5]:
# OLS Model 
# fitting the model 
model = smf.ols(formula='Preis ~ '+' + '.join(df_noPrice.columns), data = df_std).fit() 
print(model.summary()) 

# OLS Model 
# fitting the model 
model = sm.OLS(df['Preis'], sm.add_constant(df_noPrice)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     234.5
Date:                Sun, 14 Jan 2024   Prob (F-statistic):               0.00
Time:                        14:36:00   Log-Likelihood:                -7279.4
No. Observations:                1038   AIC:                         1.462e+04
Df Residuals:                    1008   BIC:                         1.477e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [6]:
train_values, test_values, train_labels, test_labels = train_test_split(df_noPrice, df['Preis'], test_size=0.01)

def del_ran(df_exog, labels, chance):
    rand_array = np.random.rand(df_exog.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    
    return [df_exog[delete_entries], labels[delete_entries], df_exog[keep_entries], labels[keep_entries]]



temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = 0.1)

In [7]:
def impute_ols(test_values, test_labels, train_values, train_labels):
    
    # Um zu vermeiden, dass die Matrix am Ende nicht mehr den vollen Rang besitzt, fügen wir einen minimalen Jitter hinzu.
    # So kann die Regression wie gehabt durchgeführt werden. Die Jitter sollte die Werte nicht nennenswert beeinflussen,
    # Aber die Matrix wird auf alle Fälle ihren vollen Range behalten.
    random_values = np.random.uniform(low=-0.001, high=0.001, size=train_values.shape)

    # Add the random values to the original DataFrame
    train_values = train_values + random_values

    # OLS Model
    # fitting the model 
    model = sm.OLS(train_labels, sm.add_constant(train_values)).fit() 

    imputed_values = model.predict(exog = sm.add_constant(test_values, has_constant='add')).tolist()
    return [np.mean((imputed_values-test_labels)**2), stats.sem(list(train_labels) + imputed_values)]

In [8]:
def impute_knn(test_values, test_labels, train_values, train_labels):
    
    tree = KDTree(train_values.values, leaf_size=5)

    imputed_values_knn_1 = []
    imputed_values_knn_3 = []
    imputed_values_knn_5 = []

    for index, entry in enumerate(test_values.values):
 
        dist, ind = tree.query([entry], k=5)
        ind = ind[0]

        current_impute_knn_1 = np.mean(train_labels.values[ind][0])
        current_impute_knn_3 = np.mean(train_labels.values[ind][:3])
        current_impute_knn_5 = np.mean(train_labels.values[ind])

        imputed_values_knn_1.append(current_impute_knn_1)
        imputed_values_knn_3.append(current_impute_knn_3)
        imputed_values_knn_5.append(current_impute_knn_5)
       
        # print(train_labels.values[ind])
        # print(current_impute_knn_1//1, current_impute_knn_3//1, current_impute_knn_5//1)
        # print(test_labels.values[index])
        # print(train_values.values[ind])

    mse_knn_1 = np.mean((test_labels.values - imputed_values_knn_1)**2)
    mse_knn_3 = np.mean((test_labels.values - imputed_values_knn_3)**2)
    mse_knn_5 = np.mean((test_labels.values - imputed_values_knn_5)**2)

    sem_knn_1 = stats.sem(list(train_labels.values)+imputed_values_knn_1)
    sem_knn_3 = stats.sem(list(train_labels.values)+imputed_values_knn_3)
    sem_knn_5 = stats.sem(list(train_labels.values)+imputed_values_knn_5)


    return [mse_knn_1, mse_knn_3, mse_knn_5, sem_knn_1, sem_knn_3, sem_knn_5]

In [12]:
def simul_knn():
    for c in np.arange(0.1, 1, 0.1):

        for i in tqdm(range(100)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_knn.at[i] = impute_knn(temp[0], temp[1], temp[2], temp[3])

        imputed_stats_knn.loc[c] = imputed_simul_knn.mean()
    imputed_stats_knn

simul_knn()
    

100%|██████████| 100/100 [00:10<00:00,  9.24it/s]


In [10]:
def simul_ols():
    for c in np.arange(0.1, 1, 0.1):

        for i in tqdm(range(100)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_ols.at[i] = impute_ols(temp[0], temp[1], temp[2], temp[3])
        imputed_stats_ols.loc[c] = imputed_simul_ols.mean()
    imputed_stats_ols

simul_ols()

100%|██████████| 100/100 [00:01<00:00, 67.82it/s]


Bei der Simulation wird irgendwann ein Fehler geworfen. Das liegt daran, dass, wenn zu viele Einträge gelöscht werden, die Matrix ihren vollen Rang verliert. Einige Dummy-Variablen sind relativ selten. Wenn nun alle Einträge mit der Dummy-Variable auf 1 (oder 0) gelöscht werden, dann wird wegen der linearen Abhängigkeit zur Konstante diese Variable gelöscht - so vermeiden wir Kollinarität.
Wenn dann aber nun im Test-Datensatz die Variable in ihrer Ausprägung 0 (oder 1) besitzt, dann können haben wir dafür keinen passenden Regressionskoeffizienten. Es werden dann 33 Variablen als Input geliefert, obwohl wir nur 32 Regressionskoeffizienten haben. Anschließend wird eine Fehlermeldung geworfen, dass der Eintrag, den wir vorhersagen wollen die falsche shape hat (32 statt 33). 
Es stellt sich zudem an diesem Punkt außerdem die Frage, wie sinnvoll es ist Variablen imputieren zu wollen, bei denen 70% der Einträge fehlen.

Bei den Auswertungen ist zu beachten, dass der wahre Standardfehler von Preis bei 23.2318 liegt.
Wir können also deutlich erkennen, dass jede Imputation der fehlenden Werte den Standardfehler künstlich verringert.

Es ist auch relativ deutlich klar, dass ein höherer k-Wert für KNN dazu führt, dass der Standardfehler weiter sinkt. Das liegt daran, dass bei einem höheren k-Wert der Mittelwert von einem größeren Teil des Datensatzes genommen wird. Dadurch wird der Mittelwert (oder zumindest Mittelwert-nahe Werte) imputiert.

In [11]:
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1)

,MSE OLS,SE OLS,MSE KNN_1,MSE KNN_3,MSE KNN_5,SE KNN_1,SE KNN_3,SE KNN_5
0.1,77337.62721,23.088734,91105.799575,74690.189792,80893.918257,23.169582,23.033303,22.961722
0.2,79310.65568,22.948537,97668.524567,80261.842916,87178.822191,23.097271,22.809229,22.661112
0.3,78089.685154,22.793268,103212.91897,87316.628853,93298.00165,23.033083,22.542148,22.350108
0.4,78476.119593,22.640244,NaN,NaN,NaN,NaN,NaN,NaN
0.5,79475.843796,22.449688,NaN,NaN,NaN,NaN,NaN,NaN
0.6,81136.03283,22.297048,NaN,NaN,NaN,NaN,NaN,NaN
0.7,54094998.633868,54.79977,NaN,NaN,NaN,NaN,NaN,NaN
0.8,203536297.501312,121.847432,NaN,NaN,NaN,NaN,NaN,NaN
0.9,1614595874.071537,612.330809,NaN,NaN,NaN,NaN,NaN,NaN
